In [ ]:
import datetime
now = datetime.datetime.now()
month = now.month
month

if 2 != 3:
    print("good")

In [ ]:
#選擇權下單範例

# 步驟
# 取得帳戶資訊
# 確立合約
# 確立order
# 送出order
 
# globals.position = globals.api.list_positions(globals.api.futopt_account)
# Offset.CLOSE: A closing offset (CO) order is a limit order that is executed at market close but can be placed anytime during the trading day. 

import shioaji as sj
from shioaji.constant import Action, OptionRight, StockPriceType, FuturesOrderType, FuturesOCType #選擇權下單，多匯入一個OptionRight常數

api = sj.Shioaji()
api.login(
    person_id='R124743829', 
    passwd='366375Dauhui'
)

#contract = globals.api.Contracts.Options[option_code] #選擇權Contract
contract = api.Contracts.Options['TXO14550L2'] #選擇權Contract
contract_snap = api.snapshots([contract])
print(contract_snap)
order = api.Order(
    action=Action.Buy,
    price=170, #價格
    quantity=2, #口數
    price_type=StockPriceType.LMT, #MKT: 市價 LMT: 限價
    order_type=FuturesOrderType.ROD,
    octype=FuturesOCType.Cover, #倉別，收盤時平倉
    OptionRight=OptionRight.Call, #選擇權類型
    account=api.futopt_account #下單帳戶指定期貨帳戶
)

#trade = api.place_order(contract, order)
#print(type(trade))


snapshots = api.snapshots([api.Contracts.Options.TXO.TXO20221214550C])
snapshots

In [ ]:
# -*- coding: utf-8 -*-

import importlib
import shioaji as sj
import login.shioaji_login as shioaji_login
from shioaji import BidAskFOPv1
# from shioaji.constant import Action, OptionRight, StockPriceType, FuturesOrderType, FuturesOCType #選擇權下單，多匯入一個OptionRight常數
importlib.reload(shioaji_login)

api = shioaji_login.login()

contract = api.Contracts.Options['TX215300L2']
optionright = sj.constant.OptionRight.Call
price = 0.5
    
order = api.Order(
    action=sj.constant.Action.Buy,
    price=price,
    quantity=1, #口數
    price_type=sj.constant.StockPriceType.LMT, #MKT: 市價 LMT: 限價
    order_type=sj.constant.FuturesOrderType.IOC, # 立即成交否則取消
    octype=sj.constant.FuturesOCType.Cover, #倉別，收盤時平倉
    OptionRight=optionright, #選擇權類型
    account=api.futopt_account #下單帳戶指定期貨帳戶
)

trade = api.place_order(contract, order)
trade.status.status

In [ ]:
import pandas as pd
if trade.status.status == 'PendingSubmit':
    print("test")
positions = api.get_account_openposition(account=api.futopt_account)
df_positions = pd.DataFrame(positions.data())
if df_positions.empty:
    print("test")